#BUILDING A CHATBOT

In [3]:
import os
from dotenv import load_dotenv
load_dotenv()

from langchain_groq import ChatGroq 
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

groq_api_key = os.getenv("GROQ_API_KEY")

model_llm = ChatGroq(
    model_name="gemma2-9b-it",
    api_key=groq_api_key,
    temperature=0.5,
    
)

In [4]:
from langchain_core.messages import HumanMessage, SystemMessage
model_llm.invoke([HumanMessage(content="Hello, how are you?")])

AIMessage(content="As an AI, I don't have feelings, but I'm here and ready to assist you! How can I help you today?\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 15, 'total_tokens': 47, 'completion_time': 0.058181818, 'prompt_time': 0.001259878, 'queue_time': 0.11149958, 'total_time': 0.059441696}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--b24e4f25-8b32-459b-9cfb-2d7a83cde1e4-0', usage_metadata={'input_tokens': 15, 'output_tokens': 32, 'total_tokens': 47})

In [6]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

model_llm.invoke(
    [
        HumanMessage(content="What is the capital of France?"),
        AIMessage(content="The capital of France is Paris."),
        HumanMessage(content="What is the capital of Germany?"),
    ]
)

AIMessage(content='The capital of Germany is Berlin. 🇩🇪  \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 40, 'total_tokens': 54, 'completion_time': 0.025454545, 'prompt_time': 0.001580188, 'queue_time': 0.088141131, 'total_time': 0.027034733}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--9077dfa8-e314-4b8e-827c-0d5fc7421a39-0', usage_metadata={'input_tokens': 40, 'output_tokens': 14, 'total_tokens': 54})

# Message History

In [8]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory


store = {}
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


In [13]:
with_message_history = RunnableWithMessageHistory(model_llm, get_session_history)

response = with_message_history.invoke(
    [
        HumanMessage(content="Hello, how are you?")
        
    ],
    config = config
)

In [11]:
config = {"configurable": {"session_id": "chat1"}}

In [14]:
response.content

"I'm doing well, thank you!  \n\nIs there anything I can help you with today? 😊  \n"

In [15]:
with_message_history.invoke(
    [
        HumanMessage(content="My name is Jeevan")
        
    ],
    config = config
)

AIMessage(content="It's nice to meet you, Jeevan!  \n\nWhat can I do for you today?\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 100, 'total_tokens': 124, 'completion_time': 0.043636364, 'prompt_time': 0.002740667, 'queue_time': 0.111655733, 'total_time': 0.046377031}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--f733fd3e-ff24-429f-90f4-b155ef3d1371-0', usage_metadata={'input_tokens': 100, 'output_tokens': 24, 'total_tokens': 124})

In [16]:
with_message_history.invoke(
    [
        HumanMessage(content="Hello, what is my name?")
        
    ],
    config = config
)

AIMessage(content='Your name is Jeevan.  \n\nI remember! 😊  \n\nIs there anything else I can help you with?\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 139, 'total_tokens': 166, 'completion_time': 0.049090909, 'prompt_time': 0.003537086, 'queue_time': 0.089164853, 'total_time': 0.052627995}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--9322d219-258d-43ef-94cc-4f1b429c8dbe-0', usage_metadata={'input_tokens': 139, 'output_tokens': 27, 'total_tokens': 166})

# Prompt Templates

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant, to the best of your ability",),
        MessagesPlaceholder(variable_name="messages"), # This is a placeholder for the messages with key, value pair
    ]
)


In [21]:
chain = prompt | model_llm

In [22]:
chain.invoke(
    [
        HumanMessage(content="Hello, how are you?")
    ]
)

AIMessage(content="As an AI, I don't have feelings like humans do. But I'm here and ready to assist you! How can I help you today? 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 38, 'prompt_tokens': 27, 'total_tokens': 65, 'completion_time': 0.069090909, 'prompt_time': 0.001404339, 'queue_time': 0.111780215, 'total_time': 0.070495248}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--6dffc2ae-d7d0-4e02-b08b-0a678a92ac1d-0', usage_metadata={'input_tokens': 27, 'output_tokens': 38, 'total_tokens': 65})

# how to invoke with chat message history

In [23]:
with_message_history = RunnableWithMessageHistory(chain, get_session_history)

In [24]:
config = {"configurable": {"session_id": "chat3"}}
response = with_message_history.invoke(
    [
        HumanMessage(content="Hello, my name is jeevan")
    ],
    config = config
)
response.content


"Hello Jeevan! It's nice to meet you. 👋\n\nHow can I help you today? 😊  \n\n"

In [25]:
## instead of 1 input variable, let's add multiple input variables

In [26]:


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant, to the best of your ability in {language}",),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model_llm

In [28]:
response = chain.invoke(
    {
        "language": "Telugu",
        "messages": [HumanMessage(content="Hello, how are you?")]
    }
)
response.content



'నమస్తే! నేను బాగున్నాను. మీలా? \n\n(Namaste! Nenu bagunnanu. Mee la?)\n\nThis translates to:\n\nHello! I am doing well. How are you?\n\n\nLet me know if you have any other questions. 😊 \n'

In [30]:
#Lets now wrap this chain in a Message History class. this time,
# because there are muliplt KEYS in the input, we need to specify the correct key to use 
# to save the chat history.

In [36]:
with_message_history = RunnableWithMessageHistory(
    chain, 
    get_session_history,
    input_messages_key="messages"
)



In [32]:
config = {"configurable": {"session_id": "chat4"}}

In [37]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="Hello, how are you?")],
        "language": "Telugu"
    },
    config = config,
)
response.content



'నాకు బాగుంది, మీకు ఏమై ఉంది? \n\n(Naku bagunndi, meeku ēmaĭ uṇḍi?)\n\nThis translates to "I\'m doing well, how are you?" in Telugu. \n\nLet me know if you have any other questions or need help with anything else. 😊\n'

# Managing CHAT message history

In [38]:
# One important concept to understand when building chatbots is how to manage
# conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the 
# Model.

# Therefore, it is important to add a step that imits the size of the messages you are passing in

In [40]:
from langchain_core.messages import SystemMessage, trim_messages
# trim_messages helper to reduce how many messages we're sending to the model.
# The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always
# keep the system message and whether to allow partial messages


In [44]:
trimmer = trim_messages(
    max_tokens=100,
    strategy="last",
    token_counter=model_llm,
    #include_system_message=True,
    allow_partial=True,
    start_on="human"
)
messages = [
    SystemMessage(content="You are a helpful assistant"),
    HumanMessage(content="Hello, how are you?"),
    AIMessage(content="I'm doing well, thank you for asking!"),
    HumanMessage(content="What is my name?"),
    AIMessage(content="Your name is Jeevan"),
    HumanMessage(content="What is my age?"),
    AIMessage(content="You are 25 years old"),
    HumanMessage(content="What is my favorite color?"),
    AIMessage(content="Your favorite color is blue"),
    HumanMessage(content="What is my favorite food?"),
    AIMessage(content="Your favorite food is pizza"),
]

trimmer.invoke(messages)

[HumanMessage(content='Hello, how are you?', additional_kwargs={}, response_metadata={}),
 AIMessage(content="I'm doing well, thank you for asking!", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='What is my name?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Your name is Jeevan', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='What is my age?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='You are 25 years old', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='What is my favorite color?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Your favorite color is blue', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='What is my favorite food?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Your favorite food is pizza', additional_kwargs={}, response_metadata={})]

In [46]:
# in a chainhow do we pass a trimmer?
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain = (
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model_llm
)

response =chain.invoke(
    {
    "messages": messages + [HumanMessage(content="What is my favorite food?")],
    "language": "Telugu"
}
)
response.content

# lets now add a new input variable to the chain



"As an AI, I don't have access to personal information about you, including your favorite food. \n\nCould you tell me what your favorite food is? 😊  \n\n"

In [47]:
response =chain.invoke(
    {
    "messages": messages + [HumanMessage(content="What is my age?")],
    "language": "Telugu"
}
)
response.content

'You are 25 years old.  \n\nI seem to remember that from before! 😊\n'

In [48]:
# lets wrap this in the message history
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

config = {"configurable": {"session_id": "chat5"}}
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="What is my age?")],
        "language": "Telugu"
    },
    config = config
)
response.content

# lets now add a new input variable to the chain


'You are 25 years old.  \n\nI seem to remember that from before! 😊  Is there anything else I can help you with?\n'